<a href="https://colab.research.google.com/github/LGBFBenchmarking/ChangeLogMaker/blob/main/Automate_differences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Test code for creating LGBF change log for Monthly refreshes. The below code find the difference between 2 dataframes/datasets.

#Load files

##new_df is new data including new year's data
##diff_df has old data (no changes or new data for the new year)

#Loads the necessary packages for the code. Important to run this in the begining of the session.

In [ ]:
import pandas as pd
import numpy as np

def get_data(url): # this function gets data stored as csv file from a given url
  df = pd.read_csv(url)
  return df


Upload the data sets for the current month and previous month. Copy the url for the raw data files for the two months and paste in respective lines here- current month is new and previous month is old. For ex. in May refresh April data is old_df and May data is new_df.

In [ ]:
url_old = "https://raw.githubusercontent.com/LGBFBenchmarking/ChangeLogMaker/refs/heads/main/LGBF_Data_May2025.csv"
url_new = "https://raw.githubusercontent.com/LGBFBenchmarking/ChangeLogMaker/refs/heads/main/LGBF_Data_June25.csv"
old_df = get_data(url_old)
new_df = get_data(url_new)

Below checks the column names - make sure they match.

In [ ]:
old_df.dtypes

,0
Code,object
LocalAuthority,object
Year,object
Indicator,float64
Numerator,float64
Denominator,float64


In [ ]:
new_df.dtypes

,0
Code,object
LocalAuthority,object
Year,object
Indicator,float64
Numerator,float64
Denominator,float64


This sets the formatting or we will have look through way more lines in the differences. So important to run these.

In [ ]:
new_df['Indicator'] = new_df['Indicator'].map("{:.2f}".format)
new_df['Numerator'] = new_df['Numerator'].map('{:.2f}'.format)
new_df['Denominator'] = new_df['Denominator'].map('{:.2f}'.format)

In [ ]:
old_df['Indicator'] = old_df['Indicator'].map('{:.2f}'.format)
old_df['Numerator'] = old_df['Numerator'].map('{:.2f}'.format)
old_df['Denominator'] = old_df['Denominator'].map('{:.2f}'.format)

In [ ]:
old_df = old_df.round(4)
new_df = new_df.round(4)

In [ ]:
old_df.head()

,Code,LocalAuthority,Year,Indicator,Numerator,Denominator
0,C&L01,Aberdeen City,2010-11,0.33,637.00,1922292.00
1,C&L01,Aberdeen City,2011-12,0.76,1554.00,2045051.00
2,C&L01,Aberdeen City,2012-13,3.64,7883.00,2163756.00
3,C&L01,Aberdeen City,2013-14,3.48,7726.00,2222588.00
4,C&L01,Aberdeen City,2014-15,3.02,7517.00,2487138.00


In [ ]:
new_df.head()

,Code,LocalAuthority,Year,Indicator,Numerator,Denominator
0,C&L01,Aberdeen City,2010-11,0.33,637.00,1922292.00
1,C&L01,Aberdeen City,2011-12,0.76,1554.00,2045051.00
2,C&L01,Aberdeen City,2012-13,3.64,7883.00,2163756.00
3,C&L01,Aberdeen City,2013-14,3.48,7726.00,2222588.00
4,C&L01,Aberdeen City,2014-15,3.02,7517.00,2487138.00


The below lines is to make sure that the original files are not messed with.

In [ ]:
df1 = old_df
df2 = new_df

This finds the new data for this month.

In [ ]:
newdf = df2[~df2.astype(str).apply(tuple, 1).isin(df1.astype(str).apply(tuple, 1))]

In [ ]:
newdf

,Code,LocalAuthority,Year,Indicator,Numerator,Denominator
6117,CHN08a,Aberdeen City,2023-24,3998.61,14347.00,69.00
6131,CHN08a,Aberdeenshire,2023-24,2591.17,10914.00,81.00
6145,CHN08a,Angus,2023-24,6224.76,10358.00,32.00
6159,CHN08a,Argyll & Bute,2023-24,2536.66,4221.00,32.00
6173,CHN08a,Clackmannanshire,2023-24,4575.44,3093.00,13.00
...,...,...,...,...,...,...
38437,CHN23,Scotland,2023-24,0.17,nan,nan
38479,CLIM03,Scotland,2023-24,28.67,157386.55,5490100.00
38486,CLIM04,Scotland,2023-24,48.15,264339.39,5490100.00
38493,CLIM05,Scotland,2023-24,56.17,308385.68,5490100.00


the below code downloads the new data into your computer as an excel file

In [ ]:
from google.colab import files
newdf. to_excel('newdf.xlsx')
files.download('newdf.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

The below code is a function to find differences. This needs to be run before the line below.

In [ ]:
def dataframe_difference(df1, df2, which=None):
    """Find rows which are different between two DataFrames."""
    comparison_df = df1.merge(
        df2,
        indicator=True,
        how='outer'
    )
    if which is None:
        diff_df = comparison_df[comparison_df['_merge'] != 'both']  # Identify data changes/updates
    else:
        diff_df = comparison_df[comparison_df['_merge'] == which]
    return diff_df

Another functions to define the column names when we merge. So run this code before the next set of lines.

In [ ]:
def getColChangedName(row):
    colsChanged = []
    for c in df1.columns.values:
        if row[c+"_diff"] == True:
            colsChanged.append(c)
    return ", ".join(colsChanged)

Same as above when we found the new data. This one is reassigning the data sets so the original data is not messed with, then merges the two data sets for comparing.

In [ ]:
df1 = old_df.set_index(['Code', 'LocalAuthority', 'Year'])
df2 = new_df.set_index(['Code','LocalAuthority', 'Year'])

diffs = df1.merge(df2, left_index=True, right_index=True, suffixes=('_old', '_new'))

diff_cols = [c+"_diff" for c in df1.columns.values]

for c in df1.columns.values:
    diffs[c+"_diff"] = diffs[c+"_old"] != diffs[c+"_new"]

diffs["Record_Changed"] = diffs[diff_cols].sum(axis=1)/len(diff_cols) > 0


diffs = diffs[diffs["Record_Changed"] == 1]
diffs["Cols_Changed"] = diffs.apply(lambda row: getColChangedName(row), axis=1)

diffs

Indicator_old Numerator_old  \
Code   LocalAuthority      Year                                  
CHN08a Aberdeen City       2023-24           nan           nan   
       Aberdeenshire       2023-24           nan           nan   
       Angus               2023-24           nan           nan   
       Argyll & Bute       2023-24           nan           nan   
       Clackmannanshire    2023-24           nan           nan   
...                                          ...           ...   
CHN08b Stirling            2023-24           nan           nan   
       West Dunbartonshire 2023-24           nan           nan   
       West Lothian        2023-24           nan           nan   
CHN08a Scotland            2023-24           nan           nan   
CHN08b Scotland            2023-24           nan           nan   

                                   Denominator_old Indicator_new  \
Code   LocalAuthority      Year                                    
CHN08a Aberdeen City       2023-24             nan       3998.61   
       Aberdeenshire       2023-24             nan       2591.17   
       Angus               2023-24             nan       6224.76   
       Argyll & Bute       2023-24             nan       2536.66   
       Clackmannanshire    2023-24             nan       4575.44   
...                                            ...           ...   
CHN08b Stirling            2023-24             nan        244.46   
       West Dunbartonshire 2023-24             nan        283.64   
       West Lothian        2023-24             nan        610.07   
CHN08a Scotland            2023-24             nan       5281.82   
CHN08b Scotland            2023-24             nan        474.68   

                                   Numerator_new Denominator_new  \
Code   LocalAuthority      Year                                    
CHN08a Aberdeen City       2023-24      14347.00           69.00   
       Aberdeenshire       2023-24      10914.00           81.00   
       Angus               2023-24      10358.00           32.00   
       Argyll & Bute       2023-24       4221.00           32.00   
       Clackmannanshire    2023-24       3093.00           13.00   
...                                          ...             ...   
CHN08b Stirling            2023-24       2339.00          184.00   
       West Dunbartonshire 2023-24       6298.00          427.00   
       West Lothian        2023-24       8724.00          275.00   
CHN08a Scotland            2023-24     363643.00         1324.00   
CHN08b Scotland            2023-24     259617.00        10518.00   

                                    Indicator_diff  Numerator_diff  \
Code   LocalAuthority      Year                                      
CHN08a Aberdeen City       2023-24            True            True   
       Aberdeenshire       2023-24            True            True   
       Angus               2023-24            True            True   
       Argyll & Bute       2023-24            True            True   
       Clackmannanshire    2023-24            True            True   
...                                            ...             ...   
CHN08b Stirling            2023-24            True            True   
       West Dunbartonshire 2023-24            True            True   
       West Lothian        2023-24            True            True   
CHN08a Scotland            2023-24            True            True   
CHN08b Scotland            2023-24            True            True   

                                    Denominator_diff  Record_Changed  \
Code   LocalAuthority      Year                                        
CHN08a Aberdeen City       2023-24              True            True   
       Aberdeenshire       2023-24              True            True   
       Angus               2023-24              True            True   
       Argyll & Bute       2023-24              True            True   
       Clackmannanshire    2023-24              True           

This downloads the differences between the two data sets there by finding any updates to previously published data. Unfortunately, due to formatting changes/rounding and sometimes lack of any data without NAs, we may see lines of data showing no changes. Simply check them and delete manually. Unfortunately, this code does pick up no changes due to strict checking. Better this than missing any updates.

In [ ]:
from google.colab import files
diffs. to_excel('diffs.xlsx')
files.download('diffs.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Use diff and newdf to get the Change log